In [653]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [654]:
import pandas as pd
books = pd.read_csv("//content/drive/MyDrive/Exam/BX-Books.csv", encoding='latin-1', delimiter=';', error_bad_lines=False)
ratings = pd.read_csv ("/content/drive/MyDrive/Exam/BX-Book-Ratings.csv",encoding='latin-1', delimiter=';', error_bad_lines=False)
users = pd.read_csv ("/content/drive/MyDrive/Exam/BX-Users.csv",encoding='latin-1', delimiter=';', error_bad_lines=False)              


<ipython-input-654-c71f8b3b76db>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv("//content/drive/MyDrive/Exam/BX-Books.csv", encoding='latin-1', delimiter=';', error_bad_lines=False)
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, s

In [655]:
#Making column names first letter be a capital letter and replace - with _

books.columns = books.columns.str.strip().str.title().str.replace('-', '_')
users.columns = users.columns.str.strip().str.title().str.replace('-', '_')
ratings.columns = ratings.columns.str.strip().str.title().str.replace('-', '_')

In [656]:
age_null = users['Age'].isnull().sum() # Sums up the 1's returned by the isnull() mask
all_users = users['User_Id'].count() # Counts the number of cells in the series (wihout the NaNs)

print(f'There are {age_null} empty age values in our set of {all_users} users (or {(age_null/all_users)*100:.2f}%).')

There are 110762 empty age values in our set of 278858 users (or 39.72%).


In [657]:
#Merging the dataframes into a new dataframe to search for the best book/country

books_by_country = pd.merge(books, ratings, on = 'Isbn', how='inner')
books_by_country = pd.merge( books_by_country, users, on ='User_Id', how = 'inner')

In [658]:
!pip install beautifulsoup4

from bs4 import BeautifulSoup

#I'm creating a function to convert HTML entities to Unicode characters

def convert_html_entities(text):
  # BeautifulSoup object with the input text and the 'html.parser' parser

    soup = BeautifulSoup(text, 'html.parser')

  #I'm using the get_text() method to extract the text without HTML tags or entities

    return soup.get_text()

books_by_country['Location'] = books_by_country['Location'].apply(convert_html_entities)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<ipython-input-658-bded17210fcf>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [659]:
#Removing the * symbol from our Location column

books_by_country['Location'] = books_by_country['Location'].str.replace('*', '')

<ipython-input-659-34ee1a2d076f>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  books_by_country['Location'] = books_by_country['Location'].str.replace('*', '')


In [660]:
#Splitting the Location column into 3 columns so the data is easier to filter and read

user_location_expanded = books_by_country['Location'].str.split(',', 2, expand=True)
user_location_expanded.columns = ['City', 'State', 'Country']
books_by_country = books_by_country.join(user_location_expanded)

<ipython-input-660-cb5200bb0a05>:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  user_location_expanded = books_by_country['Location'].str.split(',', 2, expand=True)


In [661]:
# Dropping the location column since we no longer need it

books_by_country = books_by_country.drop(column ='Location', axis = 1)

In [662]:
#We have too many Na values in our Country column

books_by_country = books_by_country['Country'].fillna('Unknown')

In [680]:
#Most liked books by Country
#top_ratings = books_by_country.sort_values('Book_Rating', ascending=False)[['Book_Title', 'Country']].head(50)
top_ratings = books_by_country.sort_values('Book_Rating', ascending=False).drop_duplicates('Country')[['Book_Title', 'Country']].head(50)
top_ratings

,Book_Title,Country
158414,The Adrian Mole Diaries,usa
938631,The Last Time They Met : A Novel,
988345,The Razor's Edge (Penguin Twentieth-Century Cl...,portugal
988386,Faust - Der Tragodie Erster Teil,germany
988295,El Hobbit,spain
287469,Hamlet (The Arden Edition of the Works of Will...,france
893127,Sex and the Single Vampire,canada
511792,God and I: A book about faith and prayer,far away...
813581,Stranger Strg Lnd,netherlands
988560,Moby Dick (Troll Illustrated Classics),iceland


In [664]:
#Checking the ratings and grouping the data in the ratings df

#This is a very important step in our data
user_groupby = ratings.groupby("User_Id") 
book_groupby = ratings.groupby("Isbn")


average_user_rating = user_groupby["Book_Rating"].mean() # This is basicly the avg number of ratings by users
number_of_ratings_by_user = user_groupby["Book_Rating"].count() # This is the number of ratings by users

average_book_rating = book_groupby["Book_Rating"].mean() # Avg book rating
number_of_book_ratings = book_groupby["Book_Rating"].count() # The number of book ratings

#Changing the name of the columns
average_user_rating.name = "Avg_rating"
number_of_ratings_by_user.name = "N_ratings"
average_book_rating.name = "Avg_rating"
number_of_book_ratings.name = "N_ratings"

In [665]:
#Merging the data into our main dataframes

users = users.join(number_of_ratings_by_user, on="User_Id")
users = users.join(average_user_rating, on="User_Id")
books = books.join(number_of_book_ratings, on="Isbn")
books = books.join(average_book_rating, on="Isbn")


In [666]:
#Cleaning the data in the N_ratings columns for our books and users dataframes
users["N_ratings"] = users["N_ratings"].fillna(0)
books["N_ratings"] = books["N_ratings"].fillna(0)

#users['N_ratings'].sort_values() -> Float, lets make it astype(int64)

#Chaging the datatype from float to int so we can work easier

users["N_ratings"] = users["N_ratings"].astype("int64")
books["N_ratings"] = books["N_ratings"].astype("int64")

In [667]:
#The location column from Users its not that important because we are running a website, not a local shop
#Clients can purchase whatever book they like. 
#We will just filter the most and best rated books with the most and best rated authors

books.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Isbn                 271360 non-null  object 
 1   Book_Title           271360 non-null  object 
 2   Book_Author          271359 non-null  object 
 3   Year_Of_Publication  271360 non-null  object 
 4   Publisher            271358 non-null  object 
 5   Image_Url_S          271360 non-null  object 
 6   Image_Url_M          271360 non-null  object 
 7   Image_Url_L          271357 non-null  object 
 8   N_ratings            271360 non-null  int64  
 9   Avg_rating           270151 non-null  float64
dtypes: float64(1), int64(1), object(8)
memory usage: 20.7+ MB


In [668]:
# Since we already have J.K Rowling as an author on our website 
# Lets see all the Harry Potter books and editions written by Rowling

books[books["Book_Title"].str.contains("Harry Potter") & books["Book_Author"].str.contains("Rowling")]

,Isbn,Book_Title,Book_Author,Year_Of_Publication,Publisher,Image_Url_S,Image_Url_M,Image_Url_L,N_ratings,Avg_rating
2143,059035342X,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books,http://images.amazon.com/images/P/059035342X.0...,http://images.amazon.com/images/P/059035342X.0...,http://images.amazon.com/images/P/059035342X.0...,571,4.900175
2809,0590353403,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,Scholastic,http://images.amazon.com/images/P/0590353403.0...,http://images.amazon.com/images/P/0590353403.0...,http://images.amazon.com/images/P/0590353403.0...,168,6.363095
3459,0439064872,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...,351,4.729345
3839,0439136350,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,197,6.467005
5431,0439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,194,6.541237
...,...,...,...,...,...,...,...,...,...,...
234611,0439294827,Harry Potter and the Sorcerer's Stone: A Delux...,J. K. Rowling,2001,Scholastic,http://images.amazon.com/images/P/0439294827.0...,http://images.amazon.com/images/P/0439294827.0...,http://images.amazon.com/images/P/0439294827.0...,2,9.000000
237863,3895849618,Harry Potter und der Gefangene von Askaban. So...,Joanne K. Rowling,2002,Dhv der HÃ¶rverlag,http://images.amazon.com/images/P/3895849618.0...,http://images.amazon.com/images/P/3895849618.0...,http://images.amazon.com/images/P/3895849618.0...,1,7.000000
251233,847888663X,Harry Potter y El Caliz de Fuego,J. K. Rowling,2001,Lidiun,http://images.amazon.com/images/P/847888663X.0...,http://images.amazon.com/images/P/847888663X.0...,http://images.amazon.com/images/P/847888663X.0...,0,NaN
257263,043955490X,Harry Potter and the Goblet of Fire (Harry Pot...,J. K. Rowling,2003,Arthur A. Levine Books,http://images.amazon.com/images/P/043955490X.0...,http://images.amazon.com/images/P/043955490X.0...,http://images.amazon.com/images/P/043955490X.0...,1,10.000000


In [669]:
#Now that we have the Avg_Rating of each book, lets find the most liked ones

books[['Book_Title', 'Book_Author','N_ratings','Avg_rating']].nlargest(1000, 'Avg_rating')

,Book_Title,Book_Author,N_ratings,Avg_rating
71,The Adventures of Drew and Ellie: The Magical ...,Charles Noland,1,10.0
81,Prescription for Terror,Sandra Levy Ceren,1,10.0
178,Der KÃ?Â¶nig in Gelb.,Raymond Chandler,1,10.0
180,Die Mars- Chroniken. Roman in ErzÃ?Â¤hlungen.,Ray Bradbury,1,10.0
184,Die Mechanismen der Freude. ErzÃ?Â¤hlungen.,Ray Bradbury,1,10.0
...,...,...,...,...
45954,The Promise,Jackie French Koller,1,10.0
45971,The Art of War: A Revised Edition of the Ellis...,Niccolo Machiavelli,1,10.0
45973,"The Violet Keystone (The Seventh Tower, Book 6)",Garth Nix,2,10.0
46350,Herbs,N. J. Lathrop,1,10.0


In [670]:

#Considering the fact that these books have a single rating and it's a 10/10, we must find the most ratings with the best rating

#These are the best rated books but they don't have that many N_Ratings

top_rated_books = books[books['Avg_rating'] > 5]
top_rated_books = top_rated_books.nlargest(100, 'N_ratings')
top_rated_books

,Isbn,Book_Title,Book_Author,Year_Of_Publication,Publisher,Image_Url_S,Image_Url_M,Image_Url_L,N_ratings,Avg_rating
5506,043935806X,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,334,5.571856
3354,0345339681,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
6330,0439136369,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,2001,Scholastic,http://images.amazon.com/images/P/0439136369.0...,http://images.amazon.com/images/P/0439136369.0...,http://images.amazon.com/images/P/0439136369.0...,226,5.345133
3839,0439136350,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,197,6.467005
2232,0812550706,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1994,Tor Books,http://images.amazon.com/images/P/0812550706.0...,http://images.amazon.com/images/P/0812550706.0...,http://images.amazon.com/images/P/0812550706.0...,195,5.302564
...,...,...,...,...,...,...,...,...,...,...
7582,0425181685,Narcissus in Chains (Anita Blake Vampire Hunte...,Laurell K. Hamilton,2001,Berkley Publishing Group,http://images.amazon.com/images/P/0425181685.0...,http://images.amazon.com/images/P/0425181685.0...,http://images.amazon.com/images/P/0425181685.0...,29,5.448276
15511,0066238501,Complete Chronicles of Narnia,C. S. Lewis,2001,Harpercollins Juvenile Books,http://images.amazon.com/images/P/0066238501.0...,http://images.amazon.com/images/P/0066238501.0...,http://images.amazon.com/images/P/0066238501.0...,29,5.862069
19879,0140071814,The Kingdom by the Sea: A Journey Around the C...,Paul Theroux,1995,Penguin Books,http://images.amazon.com/images/P/0140071814.0...,http://images.amazon.com/images/P/0140071814.0...,http://images.amazon.com/images/P/0140071814.0...,29,5.344828
65641,0316155306,The Narrows: A Novel,Michael Connelly,2004,"Little, Brown",http://images.amazon.com/images/P/0316155306.0...,http://images.amazon.com/images/P/0316155306.0...,http://images.amazon.com/images/P/0316155306.0...,29,5.517241


In [671]:
# Lets see 20 best rated books in our dataset

best_20_books = books.loc[books["N_ratings"] > 20].sort_values(by="Avg_rating", ascending=False).head(20)
best_20_books

,Isbn,Book_Title,Book_Author,Year_Of_Publication,Publisher,Image_Url_S,Image_Url_M,Image_Url_L,N_ratings,Avg_rating
3028,1844262553,Free,Paul Vincent,2003,Upfront Publishing,http://images.amazon.com/images/P/1844262553.0...,http://images.amazon.com/images/P/1844262553.0...,http://images.amazon.com/images/P/1844262553.0...,54,7.962963
27556,8445071416,El Hobbit,J. R. R. Tolkien,1991,Minotauro,http://images.amazon.com/images/P/8445071416.0...,http://images.amazon.com/images/P/8445071416.0...,http://images.amazon.com/images/P/8445071416.0...,23,7.478261
1101,0060256672,Where the Sidewalk Ends : Poems and Drawings,Shel Silverstein,1974,HarperCollins,http://images.amazon.com/images/P/0060256672.0...,http://images.amazon.com/images/P/0060256672.0...,http://images.amazon.com/images/P/0060256672.0...,33,7.121212
16190,0060248025,Falling Up,Shel Silverstein,1996,HarperCollins,http://images.amazon.com/images/P/0060248025.0...,http://images.amazon.com/images/P/0060248025.0...,http://images.amazon.com/images/P/0060248025.0...,38,6.921053
1763,0395177111,The Hobbit (Leatherette Collector's Edition),J. R. R. Tolkien,1973,Houghton Mifflin Company,http://images.amazon.com/images/P/0395177111.0...,http://images.amazon.com/images/P/0395177111.0...,http://images.amazon.com/images/P/0395177111.0...,24,6.791667
10287,0553274325,Johnny Got His Gun,Dalton Trumbo,1983,Bantam Books,http://images.amazon.com/images/P/0553274325.0...,http://images.amazon.com/images/P/0553274325.0...,http://images.amazon.com/images/P/0553274325.0...,32,6.750000
15178,0836221362,It's A Magical World: A Calvin and Hobbes Coll...,Bill Watterson,1996,Andrews McMeel Publishing,http://images.amazon.com/images/P/0836221362.0...,http://images.amazon.com/images/P/0836221362.0...,http://images.amazon.com/images/P/0836221362.0...,28,6.642857
5432,0439064864,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439064864.0...,http://images.amazon.com/images/P/0439064864.0...,http://images.amazon.com/images/P/0439064864.0...,170,6.611765
5431,0439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,http://images.amazon.com/images/P/0439139597.0...,194,6.541237
21545,1563890119,"Preludes and Nocturnes (Sandman, Book 1)",Neil Gaiman,1993,DC Comics,http://images.amazon.com/images/P/1563890119.0...,http://images.amazon.com/images/P/1563890119.0...,http://images.amazon.com/images/P/1563890119.0...,27,6.481481


In [672]:
#Lets find the most rated authors 

most_rated_authors = books.nlargest(30,'N_ratings')
most_rated_authors

# We found the most rated authors, but the most rated one has an Avg_Rating of 1.01 so we should not put his books on our website

,Isbn,Book_Title,Book_Author,Year_Of_Publication,Publisher,Image_Url_S,Image_Url_M,Image_Url_L,N_ratings,Avg_rating
26,0971880107,Wild Animus,Rich Shapero,2004,Too Far,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,2502,1.019584
408,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown",http://images.amazon.com/images/P/0316666343.0...,http://images.amazon.com/images/P/0316666343.0...,http://images.amazon.com/images/P/0316666343.0...,1295,4.468726
748,0385504209,The Da Vinci Code,Dan Brown,2003,Doubleday,http://images.amazon.com/images/P/0385504209.0...,http://images.amazon.com/images/P/0385504209.0...,http://images.amazon.com/images/P/0385504209.0...,883,4.652322
1105,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial,http://images.amazon.com/images/P/0060928336.0...,http://images.amazon.com/images/P/0060928336.0...,http://images.amazon.com/images/P/0060928336.0...,732,3.448087
522,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,http://images.amazon.com/images/P/0312195516.0...,http://images.amazon.com/images/P/0312195516.0...,http://images.amazon.com/images/P/0312195516.0...,723,4.334716
305,044023722X,A Painted House,John Grisham,2001,Dell Publishing Company,http://images.amazon.com/images/P/044023722X.0...,http://images.amazon.com/images/P/044023722X.0...,http://images.amazon.com/images/P/044023722X.0...,647,3.187017
356,0142001740,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books,http://images.amazon.com/images/P/0142001740.0...,http://images.amazon.com/images/P/0142001740.0...,http://images.amazon.com/images/P/0142001740.0...,615,4.219512
1922,067976402X,Snow Falling on Cedars,David Guterson,1995,Vintage Books USA,http://images.amazon.com/images/P/067976402X.0...,http://images.amazon.com/images/P/067976402X.0...,http://images.amazon.com/images/P/067976402X.0...,614,3.255700
118,0671027360,Angels &amp; Demons,Dan Brown,2001,Pocket Star,http://images.amazon.com/images/P/0671027360.0...,http://images.amazon.com/images/P/0671027360.0...,http://images.amazon.com/images/P/0671027360.0...,586,3.718430
706,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998,Warner Books,http://images.amazon.com/images/P/0446672211.0...,http://images.amazon.com/images/P/0446672211.0...,http://images.amazon.com/images/P/0446672211.0...,585,4.105983


In [673]:
#Lets find the best rated authors with the most N_Ratings

filtered_books = books.nlargest(60,'N_ratings')

best_rated_authors = filtered_books.groupby('Book_Author')['Avg_rating'].mean().nlargest(60).reset_index(name='Avg_rating')
best_rated_authors

#Knowing how many N_Ratings the books have, we can assume that these books are the most bought too!

,Book_Author,Avg_rating
0,Harper Lee,4.920308
1,J. K. Rowling,4.814760
2,J.D. Salinger,4.635236
3,Mitch Albom,4.543326
4,Alice Sebold,4.468726
5,MITCH ALBOM,4.406650
6,Anita Diamant,4.334716
7,Sue Monk Kidd,4.219512
8,Tracy Chevalier,4.218631
9,Dan Brown,4.185376


In [674]:
#Using API to find a description for our best_rated_authors

!pip install Wikipedia-API
import wikipediaapi
import requests


wiki_api = wikipediaapi.Wikipedia('en')


# Function to fetch author description from Wikipedia API
def fetch_author_description(author):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{author}"
    try:
        response = requests.get(url)
        response_json = response.json()
        description = response_json.get('extract')
        return description
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching description for {author}: {e}")
        return None

# Adding a new column 'Author-Description' to the dataframe
best_rated_authors['Author_Description'] = best_rated_authors['Book_Author'].apply(fetch_author_description)

best_rated_authors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,Book_Author,Avg_rating,Author_Description
0,Harper Lee,4.920308,Nelle Harper Lee was an American novelist. She...
1,J. K. Rowling,4.814760,"Joanne Rowling, best known by her pen name J. ..."
2,J.D. Salinger,4.635236,Jerome David Salinger was an American author b...
3,Mitch Albom,4.543326,"Mitchell David Albom is an American author, jo..."
4,Alice Sebold,4.468726,Alice Sebold is an American author. She is kno...
5,MITCH ALBOM,4.406650,None
6,Anita Diamant,4.334716,Anita Diamant is an American author of fiction...
7,Sue Monk Kidd,4.219512,Sue Monk Kidd is an American writer from Sylve...
8,Tracy Chevalier,4.218631,Tracy Rose Chevalier is an American-British no...
9,Dan Brown,4.185376,Daniel Gerhard Brown is an American author bes...


In [675]:
#Using a function to get the book description

def fetch_book_description(title):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"
    try:
        response = requests.get(url)
        response_json = response.json()
        description = response_json.get('extract')
        return description
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching description for {title}: {e}")
        return None

# Adding a new column 'Book-Description' to the dataframe
top_rated_books['Book_Description'] = top_rated_books['Book_Title'].apply(fetch_book_description)

print(top_rated_books)

             Isbn                                         Book_Title  \
5506   043935806X  Harry Potter and the Order of the Phoenix (Boo...   
3354   0345339681  The Hobbit : The Enchanting Prelude to The Lor...   
6330   0439136369  Harry Potter and the Prisoner of Azkaban (Book 3)   
3839   0439136350  Harry Potter and the Prisoner of Azkaban (Book 3)   
2232   0812550706      Ender's Game (Ender Wiggins Saga (Paperback))   
...           ...                                                ...   
7582   0425181685  Narcissus in Chains (Anita Blake Vampire Hunte...   
15511  0066238501                      Complete Chronicles of Narnia   
19879  0140071814  The Kingdom by the Sea: A Journey Around the C...   
65641  0316155306                               The Narrows: A Novel   
3003   0380818191                                         Truth, The   

               Book_Author Year_Of_Publication                     Publisher  \
5506         J. K. Rowling                2003         

In [676]:
#Trying another function to see if we get more descriptions for our books

def get_book_description(title):
    page = wiki_api.page(title)
    if page.exists():
        return page.summary
    else:
        return "Description not found"

# Add a new column for book descriptions
top_rated_books['Book-Description'] = top_rated_books['Book_Title'].apply(get_book_description)

# Display the updated dataframe
print(top_rated_books[['Book_Title', 'Book_Description']])

                                              Book_Title  \
5506   Harry Potter and the Order of the Phoenix (Boo...   
3354   The Hobbit : The Enchanting Prelude to The Lor...   
6330   Harry Potter and the Prisoner of Azkaban (Book 3)   
3839   Harry Potter and the Prisoner of Azkaban (Book 3)   
2232       Ender's Game (Ender Wiggins Saga (Paperback))   
...                                                  ...   
7582   Narcissus in Chains (Anita Blake Vampire Hunte...   
15511                      Complete Chronicles of Narnia   
19879  The Kingdom by the Sea: A Journey Around the C...   
65641                               The Narrows: A Novel   
3003                                          Truth, The   

              Book_Description  
5506                      None  
3354                      None  
6330                      None  
3839                      None  
2232                      None  
...                        ...  
7582                      None  
15511          

In [677]:
# get_book_description function got me some more descriptions for our Book-Description column.

top_books_desc = top_rated_books[top_rated_books['Book_Description'] != 'Description not found']
top_books_desc

,Isbn,Book_Title,Book_Author,Year_Of_Publication,Publisher,Image_Url_S,Image_Url_M,Image_Url_L,N_ratings,Avg_rating,Book_Description,Book-Description
5506,043935806X,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,334,5.571856,None,Description not found
3354,0345339681,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,281,5.007117,None,Description not found
6330,0439136369,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,2001,Scholastic,http://images.amazon.com/images/P/0439136369.0...,http://images.amazon.com/images/P/0439136369.0...,http://images.amazon.com/images/P/0439136369.0...,226,5.345133,None,Description not found
3839,0439136350,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,http://images.amazon.com/images/P/0439136350.0...,197,6.467005,None,Description not found
2232,0812550706,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1994,Tor Books,http://images.amazon.com/images/P/0812550706.0...,http://images.amazon.com/images/P/0812550706.0...,http://images.amazon.com/images/P/0812550706.0...,195,5.302564,None,Description not found
...,...,...,...,...,...,...,...,...,...,...,...,...
7582,0425181685,Narcissus in Chains (Anita Blake Vampire Hunte...,Laurell K. Hamilton,2001,Berkley Publishing Group,http://images.amazon.com/images/P/0425181685.0...,http://images.amazon.com/images/P/0425181685.0...,http://images.amazon.com/images/P/0425181685.0...,29,5.448276,None,Description not found
15511,0066238501,Complete Chronicles of Narnia,C. S. Lewis,2001,Harpercollins Juvenile Books,http://images.amazon.com/images/P/0066238501.0...,http://images.amazon.com/images/P/0066238501.0...,http://images.amazon.com/images/P/0066238501.0...,29,5.862069,None,Description not found
19879,0140071814,The Kingdom by the Sea: A Journey Around the C...,Paul Theroux,1995,Penguin Books,http://images.amazon.com/images/P/0140071814.0...,http://images.amazon.com/images/P/0140071814.0...,http://images.amazon.com/images/P/0140071814.0...,29,5.344828,None,Description not found
65641,0316155306,The Narrows: A Novel,Michael Connelly,2004,"Little, Brown",http://images.amazon.com/images/P/0316155306.0...,http://images.amazon.com/images/P/0316155306.0...,http://images.amazon.com/images/P/0316155306.0...,29,5.517241,None,Description not found
